# Magic Tools Introduction

This library provides a simple, familiar and flexible way to combine Python functions with OpenAI functions. Magic Tools wraps the OpenAI API - making it easier to manage conversations and add AI to your Python applications.

## Introduction

The `magic_tools` library enables you to maintain the state of a conversation, invoke the model for generating responses, and parse the model's responses to generate function arguments that adhere to a given function specification. It also allows you to manage function specifications and execute functions whose inputs are model-generated. In this guide, we'll walk you through the library's various functionalities and demonstrate how to utilize them effectively.

## What to expect

We'll start by introducing the basic concepts: 
1. Tool
1. Conversation
2. An Function Specification

Then, we will create a real-world application where we integrate model-generated arguments into function executions, demonstrating the interaction between the model and a SQLite database.

## What is a Tool Decorator? <a name="tool-decorator"></a>

A tool decorator from the `magic_tools` library is a Python decorator. OpenAI Functions are specified with the following fields:

- **Name:** The name of the function.
- **Description:** A description of what the function does. The model will use this to decide when to call the function.
- **Parameters:** The parameters object contains all of the input fields the function requires. These inputs can be of the following types: String, Number, Boolean, Object, Null, AnyOf. Refer to the [API reference docs](https://platform.openai.com/docs/api-reference/chat) for details.
- **Required:** Which of the parameters are required to make a query. The rest will be treated as optional.

These are auto-generated from a Python function when annotated with a `@tool`. 

In [11]:
# !pip install agentai

In [12]:
%load_ext autoreload
%autoreload 2

GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Utilities

First let's import a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [13]:
from agentai.api import chat_complete, chat_complete_execute_fn
from agentai.openai_function import tool, ToolRegistry
from agentai.conversation import Conversation
from agentai.sqlite_utils import DBUtils

from typing import Any
from enum import Enum
import sqlite3


weather_registry = ToolRegistry()  # Namespace for functions

## Basic concepts

Next we'll create a specification for a function called ```get_current_weather```. Later we'll pass this function specification to the API in order to generate function arguments that adhere to the specification.

### Tool

`@tool` is a magic decorator that wraps a Python function and generates a function specification using the function's docstring and type hints. 

In [14]:
class TemperatureUnit(Enum):
    celsius = "celsius"
    fahrenheit = "fahrenheit"


@tool(weather_registry)
def get_current_weather(location: str, format: TemperatureUnit) -> str:
    """
    Get the current weather

    Args:
        location (str): The city and state, e.g. San Francisco, CA
        format (str): The temperature unit to use. Infer this from the users location.

    Returns:
        str: The current weather
    """
    # Your function implementation goes here.
    return ""

### Conversation

Conversation is a class that maintains the state of a conversation. It is used to keep track of the conversation history. 

In [15]:
conversation = Conversation()
conversation.add_message(
    role="system",
    content="""You are an AI assistant.
- Follow the user's requirements carefully & to the letter.
- First think step-by-step - describe your plan for what to do, written out in great detail.
- Ask the user for any missing information.
- Then decide about functions to use.
- Minimize any other statements.""",
)

conversation.add_message("user", "what is the weather like today?")

chat_response = chat_complete(
    conversation=conversation,
    tool_registry=weather_registry,
    model=GPT_MODEL,
    return_function_params=False,
)
message = chat_response["choices"][0]["message"]  # Get the first message
conversation.add_message(message["role"], message["content"])
message

2023-06-15 19:35:34.292 | INFO     | agentai.api:chat_complete:49 - OpenAI API returned: {
  "role": "assistant",
  "content": "Sure, I can help you with that. Could you please provide me with the location?"
}
2023-06-15 19:35:34.294 | INFO     | agentai.api:chat_complete:60 - Got Message: {
  "id": "chatcmpl-7RhtN0s4VzKpF193RWuHjjaWdZeSJ",
  "object": "chat.completion",
  "created": 1686837933,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Sure, I can help you with that. Could you please provide me with the location?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 154,
    "completion_tokens": 19,
    "total_tokens": 173
  }
}


<OpenAIObject at 0x128719630> JSON: {
  "role": "assistant",
  "content": "Sure, I can help you with that. Could you please provide me with the location?"
}

In [16]:
# Once the user provides the required information, the model can generate the function arguments
conversation.add_message("user", "I'm in Bengaluru, India")
chat_response = chat_complete(
    conversation=conversation,
    tool_registry=weather_registry,
    model=GPT_MODEL,
    return_function_params=True,
)

2023-06-15 19:35:36.511 | INFO     | agentai.api:chat_complete:49 - OpenAI API returned: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Bengaluru, India\",\n  \"format\": \"celsius\"\n}"
  }
}
2023-06-15 19:35:36.512 | INFO     | agentai.api:chat_complete:54 - Got Function Call: {
  "id": "chatcmpl-7RhtPsYKOrbIfvO2AZ0jjFGC7zbPu",
  "object": "chat.completion",
  "created": 1686837935,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Bengaluru, India\",\n  \"format\": \"celsius\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 187,
    "completion_tokens": 29,
    "total_tokens": 216
  }
}


## Integrating API calls with function execution

In our next example, we'll demonstrate how to execute functions whose inputs are model-generated, and use this to implement an agent that can answer questions for us about a database. For simplicity we'll use the [Chinook sample database](https://www.sqlitetutorial.net/sqlite-sample-database/).

*Note:* SQL generation use cases are high-risk in a production environment - models can be unreliable when generating consistent SQL syntax. A more reliable way to solve this problem may be to build a query generation API that takes the desired columns as input from the model.

### Pull SQL Database Info

First let's define some helpful utility functions to extract data from a SQLite database.

In [17]:
conn = sqlite3.connect("../data/Chinook.db")
print("Opened database successfully")

Opened database successfully


Now can use these utility functions to extract a representation of the database schema.

In [18]:
db_registry = ToolRegistry()  # Namespace for functions
database_string = DBUtils(conn).get_database_string()
print(database_string)

Table: Album
Columns: AlbumId, Title, ArtistId
Table: Artist
Columns: ArtistId, Name
Table: Customer
Columns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
Table: Employee
Columns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
Table: Genre
Columns: GenreId, Name
Table: Invoice
Columns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
Table: InvoiceLine
Columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
Table: MediaType
Columns: MediaTypeId, Name
Table: Playlist
Columns: PlaylistId, Name
Table: PlaylistTrack
Columns: PlaylistId, TrackId
Table: Track
Columns: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice


As before, we'll define a function specification for the function we'd like the API to generate arguments for. Notice that we are inserting the database schema into the function specification. This will be important for the model to know about.

In [19]:
@tool(registry=db_registry)
def ask_database(query: str) -> Any:
    """
    Use this function to answer user questions about music. Input should be a fully formed SQL query.

    Args:
        query (str):SQL query extracting info to answer the user's question.
                    SQL should be written using this database schema:
                    Table: Album
                    Columns: AlbumId, Title, ArtistId
                    Table: Artist
                    Columns: ArtistId, Name
                    Table: Customer
                    Columns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
                    Table: Employee
                    Columns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
                    Table: Genre
                    Columns: GenreId, Name
                    Table: Invoice
                    Columns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
                    Table: InvoiceLine
                    Columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
                    Table: MediaType
                    Columns: MediaTypeId, Name
                    Table: Playlist
                    Columns: PlaylistId, Name
                    Table: PlaylistTrack
                    Columns: PlaylistId, TrackId
                    Table: Track
                    Columns: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice

                    IMPORTANT: Please return a fixed SQL in PLAIN TEXT.
                    Your response should consist of ONLY the SQL query.
    """
    try:
        results = conn.execute(query).fetchall()
        return results
    except Exception as e:
        raise Exception(f"SQL error: {e}")

### SQL Execution

Now let's implement the function that the agent will use to query the database. We also need to implement utilities to integrate the calls to the Chat Completions API with the function it is calling.

In [25]:
agent_system_message = """You are ChinookGPT, a helpful assistant who gets answers to user questions from the Chinook Music Database.
Provide as many details as possible to your users
Begin!"""

sql_conversation = Conversation()
sql_conversation.add_message(role="system", content=agent_system_message)
sql_conversation.add_message(
    role="user", content="Hi, who are the top 5 artists by number of tracks"
)
results, executed_function = chat_complete_execute_fn(
    conversation=sql_conversation,
    tool_registry=db_registry,
    model=GPT_MODEL,
)
print(results)
sql_conversation.add_message(role="assistant", content=str(results), name=executed_function.__name__)

2023-06-15 19:39:46.473 | INFO     | agentai.api:chat_complete:44 - OpenAI API returned: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "ask_database",
    "arguments": "{\n  \"query\": \"SELECT Artist.Name, COUNT(*) AS TrackCount FROM Artist JOIN Album ON Artist.ArtistId = Album.ArtistId JOIN Track ON Album.AlbumId = Track.AlbumId GROUP BY Artist.ArtistId ORDER BY TrackCount DESC LIMIT 5;\"\n}"
  }
}
2023-06-15 19:39:46.473 | INFO     | agentai.api:chat_complete:49 - Got Function Call: {
  "id": "chatcmpl-7RhxQkzMzo5AzxWH5OHRmsf28Uzx3",
  "object": "chat.completion",
  "created": 1686838184,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "ask_database",
          "arguments": "{\n  \"query\": \"SELECT Artist.Name, COUNT(*) AS TrackCount FROM Artist JOIN Album ON Artist.ArtistId = Album.ArtistId JOIN Track ON Album.

[('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]


In [26]:
sql_conversation.display_conversation()

system: You are ChinookGPT, a helpful assistant who gets answers to user questions from the Chinook Music Database.
Provide as many details as possible to your users
Begin!


user: Hi, who are the top 5 artists by number of tracks


assistant: [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]




In [27]:
sql_conversation.add_message(
    "user", "What is the name of the album with the most tracks"
)
chat_response = chat_complete_execute_fn(
    conversation=sql_conversation,
    tool_registry=db_registry,
    model=GPT_MODEL,
    callable_function=ask_database,
)

TypeError: chat_complete_execute_fn() got an unexpected keyword argument 'function_registry'

In [ ]:
sql_conversation.display_conversation()

# Using Multiple Functions

In [ ]:
from pathlib import Path

data_dir = Path("../data/papers")
data_dir.mkdir(exist_ok=True, parents=True)
assert data_dir.exists()

Path.ls = lambda x: list(x.iterdir())

In [ ]:
df = 